In [22]:
import pandas as pd
from glob import glob
import os
import numpy as np

In [23]:
files = glob('C:\\Users\\jlandman\\Desktop\\newData\\SJ_AQ_Navarro\\final\\*_Johannes.xlsx')
print(files)

['C:\\Users\\jlandman\\Desktop\\newData\\SJ_AQ_Navarro\\final\\GlaThiDa_Navarro1_Johannes.xlsx', 'C:\\Users\\jlandman\\Desktop\\newData\\SJ_AQ_Navarro\\final\\GlaThiDa_Navarro2_Johannes.xlsx']


In [24]:
thin_dict = {"ELFENBEINBREEN": 14,
             "SVEIGBREEN": 4,
}

In [25]:
data_new = pd.DataFrame([])

for file in files:
    data = pd.read_excel(file, sheetname='TTT - GL. THICKNESS POINT DATA')
    for gid in np.unique(data.GlaThiDa_ID.values):
        temp = data[data.GlaThiDa_ID==gid]

        gname = temp[temp.GlaThiDa_ID == gid].GLACIER_NAME.values[0]
        pol_u = temp[temp.GlaThiDa_ID == gid].POLITICAL_UNIT.values[0]
        surv_date = temp[temp.GlaThiDa_ID == gid].SURVEY_DATE.values[0]
        remarks = temp[temp.GlaThiDa_ID == gid].REMARKS.iloc[0]
        gtd_id = gid

        temp = temp.groupby(temp.index // thin_dict[gname]).mean() # average over every x values as specified above

        temp['POLITICAL_UNIT'] = pol_u
        temp['GLACIER_NAME'] = gname
        temp['REMARKS'] = remarks
        temp['DATA_FLAG'] = np.nan
        temp['GlaThiDa_ID'] = gtd_id
        temp['SURVEY_DATE'] = surv_date
        temp['POINT_ID'] = range(1,len(temp)+1)


        data_new = data_new.append(temp, ignore_index=True)

data_new['THICKNESS_UNCERTAINTY'] = data_new['THICKNESS_UNCERTAINTY'].map('{:.4f}'.format)
data_new['THICKNESS'] = data_new['THICKNESS'].map('{:.1f}'.format)

data_new = data_new[['GlaThiDa_ID','POLITICAL_UNIT', 'GLACIER_NAME', 'SURVEY_DATE', 'POINT_ID',  'POINT_LAT',  'POINT_LON',  'ELEVATION', 
                     'THICKNESS', 'THICKNESS_UNCERTAINTY', 'DATA_FLAG', 'REMARKS']]

data_new = data_new.sort_values(by=['GlaThiDa_ID','POINT_ID'])
data_new.to_excel(os.path.join(os.path.dirname(file),file.split('.')[0]+'_thinned.xlsx'), index=False)